In [ ]:
#https://medium.com/analytics-vidhya/transforming-the-world-into-paintings-with-cyclegan-6748c0b85632


In [5]:
import tensorflow_datasets as tfds

data, metadata = tfds.load('cycle_gan/monet2photo', with_info=True, as_supervised=True)
train_x, train_y, test_x, test_y = data['trainA'], data['trainB'], data['testA'], data['testB']


In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal

epochs = 50

LAMBDA = 10

img_rows, img_cols, channels = 256, 256, 3
weight_initializer = RandomNormal(stddev=0.02)

gen_g_optimizer = gen_f_optimizer = Adam(lr=0.0002, beta_1=0.5)
dis_x_optimizer = dis_y_optimizer = Adam(lr=0.0002, beta_1=0.5)

/Users/billyluqiu/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
import tensorflow as tf

def preprocess_image(image, _):
    return tf.reshape(tf.cast(tf.image.resize(image, (int(img_rows), int(img_cols))), tf.float32) / 127.5 - 1, (1, img_rows, img_cols, channels))


In [9]:
train_x = train_x.map(preprocess_image)
train_y = train_y.map(preprocess_image)
test_x = test_x.map(preprocess_image)
test_y = test_y.map(preprocess_image)


In [10]:
train_x

<MapDataset shapes: (1, 256, 256, 3), types: tf.float32>

In [11]:
from tensorflow.keras.layers import Conv2D, LeakyReLU
from tensorflow_addons.layers import InstanceNormalization

def Ck(input, k, use_instancenorm=True):
    block = Conv2D(k, (4, 4), strides=2, padding='same', kernel_initializer=weight_initializer)(input)
    if use_instancenorm:
        block = InstanceNormalization(axis=-1)(block)
    block = LeakyReLU(0.2)(block)

    return block

/Users/billyluqiu/miniforge3/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [12]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

def discriminator():
    dis_input = Input(shape=(img_rows, img_cols, channels))
    
    d = Ck(dis_input, 64, False)
    d = Ck(d, 128)
    d = Ck(d, 256)
    d = Ck(d, 512)

    d = Conv2D(1, (4, 4), padding='same', kernel_initializer=weight_initializer)(d)
    
    dis = Model(dis_input, d)
    dis.compile(loss='mse', optimizer=dis_x_optimizer)
    return dis